In [1]:
# check GPU version tf is installed
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 不显示等级2以下的提示信息
print('GPU', tf.test.is_gpu_available())

C:\Utilities\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


GPU True


In [2]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
# load data: train, test
(ox_train,oy_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

In [4]:
ox_train.shape,oy_train.shape

((60000, 28, 28), (60000,))

In [5]:
# split data:  train->train,valid
x_train,x_valid,y_train,y_valid=train_test_split(ox_train,oy_train,test_size=0.2,random_state=111)

In [6]:
x_train.shape,y_train.shape

((48000, 28, 28), (48000,))

In [7]:
x_train.max(),y_train.max()

(255, 9)

In [8]:
# Normalization

x_avg = np.mean(x_train,axis=0,keepdims=True)  # the axis contains 48000 
x_train = x_train-x_avg
x_norm = np.max(np.abs(x_train),axis=0,keepdims=True)
x_norm[x_norm==0]=1 # when x_norm = 0
x_train = x_train/x_norm

x_valid=(x_valid-x_avg)/x_norm
x_test=(x_test-x_avg)/x_norm

# x_train=np.expand_dims(x_train,axis=-1)
# x_valid=np.expand_dims(x_valid,axis=-1)
# x_test=np.expand_dims(x_test,axis=-1)

In [9]:
# model

lin = tf.keras.layers.Input(shape=(28,28))
lact = lin
lact = tf.keras.layers.Flatten()(lact)  
# lact = tf.keras.layers.Dense(150,activation='relu')(lact)
lact = tf.keras.layers.Dense(60,activation='relu')(lact)
lact = tf.keras.layers.Dropout(0.25)(lact)
lact = tf.keras.layers.Dense(10,activation='softmax')(lact)      
lout = lact

model = tf.keras.Model(lin,lout)
model.summary()
model.compile(tf.keras.optimizers.Adam(learning_rate=1e-3),
              tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.categorical_accuracy])

from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)


logs_base_dir = 'C:\workspaceWin\DataScience\ISMINDeepLearning\logs'
%load_ext tensorboard
cb=list()
cb.append(tf.keras.callbacks.TensorBoard(log_dir=logs_base_dir))


history=model.fit(x_train,y_train,validation_data=(x_valid,y_valid),epochs=20,callbacks=cb)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 60)                47100     
_________________________________________________________________
dropout (Dropout)            (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                610       
Total params: 47,710
Trainable params: 47,710
Non-trainable params: 0
_________________________________________________________________
Train on 48000 samples, validate on 12000 samples


Using TensorFlow backend.


Epoch 1/20
48000/48000 [==============================] - 4s 91us/sample - loss: 0.3807 - categorical_accuracy: 0.8876 - val_loss: 0.1902 - val_categorical_accuracy: 0.9437
Epoch 2/20
48000/48000 [==============================] - 3s 65us/sample - loss: 0.2197 - categorical_accuracy: 0.9357 - val_loss: 0.1541 - val_categorical_accuracy: 0.9559
Epoch 3/20
48000/48000 [==============================] - 3s 67us/sample - loss: 0.1828 - categorical_accuracy: 0.9441 - val_loss: 0.1388 - val_categorical_accuracy: 0.9592
Epoch 4/20
48000/48000 [==============================] - 3s 66us/sample - loss: 0.1618 - categorical_accuracy: 0.9504 - val_loss: 0.1268 - val_categorical_accuracy: 0.9627
Epoch 5/20
48000/48000 [==============================] - 3s 67us/sample - loss: 0.1460 - categorical_accuracy: 0.9543 - val_loss: 0.1199 - val_categorical_accuracy: 0.9659
Epoch 6/20
48000/48000 [==============================] - 3s 66us/sample - loss: 0.1352 - categorical_accuracy: 0.9578 - val_loss: 0.11

In [10]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
acc=100*np.sum(pred==y_test)/x_test.shape[0]
acc

97.11

In [13]:
logs_base_dir = 'C:/workspaceWin/DataScience/ISMINDeepLearning/logs/'
#%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir {logs_base_dir} --port=6060
# %kill 16928

Reusing TensorBoard on port 6060 (pid 4176), started 0:01:00 ago. (Use '!kill 4176' to kill it.)